In [51]:
import os
import numpy as np
import pandas as pd

base_path="/Users/christophenoblanc/Documents/ProjetsPython/DSSP_Projet_DVF"
folder_path=os.path.join("data_dvf")
pd.set_option('display.max_columns', 500)

In [46]:
def read_dvf_from_csv(file_path=""):
    df=pd.read_csv(file_path,
               sep=',',decimal=".",encoding="utf_8",dtype=np.str,parse_dates=['date_mutation'])
    
    # Remove les 'Dépendance' : on ne garde que ce qui n'est pas une dépendance
    #df=df[df['type_local'] != 'Dépendance']
    return(df)

def convert_dvf(df):
    # remove some columns and records
#    df.drop(['ancien_code_commune','ancien_nom_commune','ancien_id_parcelle'
#            ,'lot1_numero','lot2_numero','lot3_numero','lot4_numero','lot5_numero']
#        , axis=1, inplace=True)
    
    # Convert numeric values
    df['valeur_fonciere']=pd.to_numeric(df['valeur_fonciere'].replace('NaN',''), errors='raise')
    df['nombre_lots']=pd.to_numeric(df['nombre_lots'].replace('NaN',''), errors='raise')
    df['code_type_local']=pd.to_numeric(df['code_type_local'].replace('NaN',''), errors='raise')
    df['surface_reelle_bati']=pd.to_numeric(df['surface_reelle_bati'].replace('NaN',''), errors='raise')
    df['nombre_pieces_principales']=pd.to_numeric(df['nombre_pieces_principales'].replace('NaN',''), errors='raise')
    df['surface_terrain']=pd.to_numeric(df['surface_terrain'].replace('NaN',''), errors='raise')
    df['longitude']=pd.to_numeric(df['longitude'].replace('NaN',''), errors='raise')
    df['latitude']=pd.to_numeric(df['latitude'].replace('NaN',''), errors='raise')
    df['lot1_surface_carrez'] =pd.to_numeric(df['lot1_surface_carrez'], errors='raise').fillna(0)
    df['lot2_surface_carrez']=pd.to_numeric(df['lot2_surface_carrez'], errors='raise').fillna(0)
    df['lot3_surface_carrez']=pd.to_numeric(df['lot3_surface_carrez'], errors='raise').fillna(0)
    df['lot4_surface_carrez']=pd.to_numeric(df['lot4_surface_carrez'], errors='raise').fillna(0)
    df['lot5_surface_carrez']=pd.to_numeric(df['lot5_surface_carrez'], errors='raise').fillna(0)
    # Add a new feature (surface carrez totale)
    df['surface_carrez_totale']=df['lot1_surface_carrez']+df['lot2_surface_carrez'] \
        +df['lot3_surface_carrez']+df['lot4_surface_carrez']+df['lot5_surface_carrez']
    # Remove records with NULL Target value
    df.dropna(axis=0, subset=['valeur_fonciere'], inplace=True) # remove records which we do not know the price
    # remove detailled surfaces by lots as most are zero (NULL)
    df.drop(['lot1_surface_carrez','lot2_surface_carrez','lot3_surface_carrez'
             ,'lot4_surface_carrez','lot5_surface_carrez'], axis=1, inplace=True)
    
    # Add the count of mutations
    print("Add count of mutations records")
    count_mutations=df[['id_mutation','date_mutation']].groupby(['id_mutation']).count()
    df=pd.merge(df, count_mutations, left_on='id_mutation', right_on='id_mutation',how='left')
    df = df.rename(columns = {'date_mutation_x':'date_mutation','date_mutation_y':'mutation_count'})
    df['mutation_count'].fillna(0,inplace=True)
    
    print("Add count of Maisons records")
    Maison=df[df['type_local'] == 'Maison'].copy()
    count_maisons=Maison[['id_mutation','date_mutation']].groupby(['id_mutation']).count()
    df=pd.merge(df, count_maisons, left_on='id_mutation', right_on='id_mutation',how='left')
    df = df.rename(columns = {'date_mutation_x':'date_mutation','date_mutation_y':'maisons_count'})
    df['maisons_count'].fillna(0,inplace=True)
    
    print("Add count of Appartement records")
    Appartement=df[df['type_local'] == 'Appartement']
    count_appartement=Appartement[['id_mutation','date_mutation']].groupby(['id_mutation']).count()
    df=pd.merge(df, count_appartement, left_on='id_mutation', right_on='id_mutation',how='left')
    df = df.rename(columns = {'date_mutation_x':'date_mutation','date_mutation_y':'appartements_count'})
    df['appartements_count'].fillna(0,inplace=True)
    
    print("Add count of Dependance records")
    Dependance=df[df['type_local'] == 'Dépendance']
    count_dependance=Dependance[['id_mutation','date_mutation']].groupby(['id_mutation']).count()
    df=pd.merge(df, count_dependance, left_on='id_mutation', right_on='id_mutation',how='left')
    df = df.rename(columns = {'date_mutation_x':'date_mutation','date_mutation_y':'dependances_count'})
    df['dependances_count'].fillna(0,inplace=True)
    
    print("Add count of Local Commercial records")
    LocalCommercial=df[df['type_local'] == 'Local industriel. commercial ou assimilé']
    count_commerce=LocalCommercial[['id_mutation','date_mutation']].groupby(['id_mutation']).count()
    df=pd.merge(df, count_commerce, left_on='id_mutation', right_on='id_mutation',how='left')
    df = df.rename(columns = {'date_mutation_x':'date_mutation','date_mutation_y':'commerces_count'})
    df['commerces_count'].fillna(0,inplace=True)
    
    return df

In [47]:
#df_oneyear=read_dvf_from_csv(file_path=base_path+"/data_dvf/2018_dvf_full.csv")

In [48]:
#df_oneyear[df_oneyear['type_local']=='Maison']['lot1_surface_carrez'].value_counts()
#df['lot1_surface_carrez'] =pd.to_numeric(df['lot1_surface_carrez'].str.replace(',','.'), errors='raise').fillna(0)

#Maisons=df_oneyear[df_oneyear['type_local']=='Maison'].copy()
#Maisons=df_oneyear.copy()
#Maisons['test_num']=pd.to_numeric(Maisons['lot1_surface_carrez'], errors='raise')

In [49]:
#list_of_years={"2014","2015","2016","2017","2018"}
list_of_years={"2018"}
for year in list_of_years:
    print("start reading csv for year="+year)
    df_oneyear=read_dvf_from_csv(file_path=base_path+"/data_dvf/"+year+"_dvf_full.csv")
    print("start converting...")
    df_oneyear=convert_dvf(df_oneyear)
    print("save converted DataFrame...")
    df_oneyear.to_pickle(folder_path+'/DataFrame_'+year)


start reading csv for year=2018
start converting...
Add count of mutations records
Add count of Maisons records
Add count of Appartement records
Add count of Dependance records
Add count of Local Commercial records
save converted DataFrame...


In [43]:
#df_oneyear.info()

In [12]:
df_oneyear['type_local'].value_counts()

Maison                                      477602
Appartement                                 365812
Dépendance                                  279787
Local industriel. commercial ou assimilé     83010
Name: type_local, dtype: int64

In [ ]:
df_oneyear[df_oneyear["id_mutation"]=="2018-2"]   

In [68]:
df_oneyear[18:26]

,id_mutation,date_mutation,numero_disposition,nature_mutation,valeur_fonciere,adresse_numero,adresse_suffixe,adresse_nom_voie,adresse_code_voie,code_postal,code_commune,nom_commune,code_departement,id_parcelle,numero_volume,nombre_lots,code_type_local,type_local,surface_reelle_bati,nombre_pieces_principales,code_nature_culture,nature_culture,code_nature_culture_speciale,nature_culture_speciale,surface_terrain,longitude,latitude,surface_carrez_totale,mutation_count,maisons_count,appartements_count,dependances_count,commerces_count
18,2018-11,2018-01-08,000001,Vente,47700.0,NaN,NaN,LE LAIT,B064,01240,01211,Lent,01,012110000A0387,NaN,0,NaN,NaN,NaN,NaN,E,eaux,ETANG,Etangs,157020.0,5.209132,46.123578,0.0,8,1.0,0.0,1.0,0.0
19,2018-11,2018-01-08,000001,Vente,47700.0,NaN,NaN,LE LAIT,B064,01240,01211,Lent,01,012110000A0388,NaN,0,NaN,NaN,NaN,NaN,L,landes,NaN,NaN,2770.0,5.208416,46.125772,0.0,8,1.0,0.0,1.0,0.0
20,2018-11,2018-01-08,000001,Vente,47700.0,NaN,NaN,LE LAIT,B064,01240,01211,Lent,01,012110000A0389,NaN,0,NaN,NaN,NaN,NaN,BS,taillis sous futaie,NaN,NaN,2120.0,5.207920,46.125619,0.0,8,1.0,0.0,1.0,0.0
21,2018-11,2018-01-08,000001,Vente,47700.0,NaN,NaN,LE LAIT,B064,01240,01211,Lent,01,012110000A0390,NaN,0,NaN,NaN,NaN,NaN,L,landes,NaN,NaN,3740.0,5.207477,46.124555,0.0,8,1.0,0.0,1.0,0.0
22,2018-11,2018-01-08,000001,Vente,47700.0,800,NaN,RTE DE SEILLON,0215,01240,01211,Lent,01,012110000A0401,NaN,0,1.0,Maison,225.0,5.0,S,sols,NaN,NaN,640.0,5.205776,46.123353,0.0,8,1.0,0.0,1.0,0.0
23,2018-11,2018-01-08,000001,Vente,47700.0,800,NaN,RTE DE SEILLON,0215,01240,01211,Lent,01,012110000A0401,NaN,0,3.0,Dépendance,NaN,0.0,S,sols,NaN,NaN,640.0,5.205776,46.123353,0.0,8,1.0,0.0,1.0,0.0
24,2018-11,2018-01-08,000001,Vente,47700.0,NaN,NaN,LE LAIT,B064,01240,01211,Lent,01,012110000A0402,NaN,0,NaN,NaN,NaN,NaN,P,prés,NaN,NaN,12790.0,5.205758,46.122967,0.0,8,1.0,0.0,1.0,0.0
25,2018-11,2018-01-08,000001,Vente,47700.0,NaN,NaN,LE LAIT,B064,01240,01211,Lent,01,012110000A0405,NaN,0,NaN,NaN,NaN,NaN,T,terres,NaN,NaN,3820.0,5.208939,46.121088,0.0,8,1.0,0.0,1.0,0.0
